In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poetry/Kanye_West.txt
/kaggle/input/poetry/johnny-cash.txt
/kaggle/input/poetry/kanye-west.txt
/kaggle/input/poetry/bruno-mars.txt
/kaggle/input/poetry/dickinson.txt
/kaggle/input/poetry/amy-winehouse.txt
/kaggle/input/poetry/blink-182.txt
/kaggle/input/poetry/paul-simon.txt
/kaggle/input/poetry/patti-smith.txt
/kaggle/input/poetry/bieber.txt
/kaggle/input/poetry/disney.txt
/kaggle/input/poetry/jimi-hendrix.txt
/kaggle/input/poetry/lin-manuel-miranda.txt
/kaggle/input/poetry/adele.txt
/kaggle/input/poetry/dj-khaled.txt
/kaggle/input/poetry/beatles.txt
/kaggle/input/poetry/r-kelly.txt
/kaggle/input/poetry/lady-gaga.txt
/kaggle/input/poetry/radiohead.txt
/kaggle/input/poetry/britney-spears.txt
/kaggle/input/poetry/alicia-keys.txt
/kaggle/input/poetry/rihanna.txt
/kaggle/input/poetry/joni-mitchell.txt
/kaggle/input/poetry/dolly-parton.txt
/kaggle/input/poetry/drake.txt
/kaggle/input/poetry/Lil_Wayne.txt
/kaggle/input/poetry/notorious_big.txt
/kaggle/input/poetry/eminem.txt
/

In [2]:
import tensorflow as tf
import keras
import re
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, SimpleRNN, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences

# GOAL - create a bot to generate song lyrics using RNN

In [3]:
chars = list('abcdefghijklmnopqrstuvwxyz')
chars.append(' ')
chars.append('\n')
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_window = 20
step = 3

In [4]:
def create_model(artist):
    
    with open('/kaggle/input/poetry/'+artist+'.txt') as f:
        text = f.read().lower()

    text = re.sub('[^a-z \n]','',text)

    inputs = []
    next_chars = []

    for i in range(0,len(text)-char_window,step):
        inputs.append(text[i:i+char_window])
        next_chars.append(text[i+char_window])

    pairs = zip(inputs,next_chars)
    list(pairs)[:100]

    X = np.zeros((len(inputs),char_window),dtype=np.float32)

    for i in range(len(inputs)):
        for j in range(char_window):
            X[i,j] = char_to_index[inputs[i][j]]

    y = np.zeros(len(next_chars),dtype=np.float32)

    for i in range(len(next_chars)):
        y[i] = char_to_index[next_chars[i]]

    y = to_categorical(y)
    
    model = Sequential()

    model.add(Embedding(28,64,input_length=char_window))
    model.add(LSTM(256,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(28,activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    model.fit(X,y,epochs=5,batch_size=20)
    model.save('/kaggle/working/'+artist)
    return model

In [5]:
kanye = create_model('Kanye_West')

2022-03-15 04:12:42.077515: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-03-15 04:12:42.645061: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
4183/4183 [==============================] - 235s 55ms/step - loss: 2.2106 - accuracy: 0.3450
Epoch 2/5
4183/4183 [==============================] - 232s 56ms/step - loss: 1.7951 - accuracy: 0.4474
Epoch 3/5
4183/4183 [==============================] - 230s 55ms/step - loss: 1.6579 - accuracy: 0.4823
Epoch 4/5
4183/4183 [==============================] - 236s 57ms/step - loss: 1.5642 - accuracy: 0.5086
Epoch 5/5
4183/4183 [==============================] - 234s 56ms/step - loss: 1.4948 - accuracy: 0.5277


2022-03-15 04:32:13.316122: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [6]:
def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def temp_scale(conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

def generate_line(model,context,temp):
    line = context
    next_char = ''
    counter = 0
    while counter<100 and next_char != '\n':
        X = [[char_to_index[line[i]] for i in range(0,len(line))]]
        X = pad_sequences(X,maxlen=char_window)
        y = model.predict(X)
        next_char = index_to_char[temp_scale(y[0,:],temp)]
        counter+=1
        line+=next_char
    return line

In [7]:
temps = [.01, 0.2, 0.5, 1.0, 1.2, 2]

for temp in temps:
    print('\n\n')
    print(generate_line(kanye,'i had a dream i could',temp))




i had a dream i could be the best a stop to the chool but i want to the stop to the chool but i want to the stop to the c



i had a dream i could be the best a baby the best a little best a starting the baby to the stop and i want to the chool b



i had a dream i could be on the hood of the day realy with my famous some to we could be all coloned in the crashie




i had a dream i could dome sayion




i had a dream i could be the tapan




i had a dream i couldb yge



In [8]:
for temp in temps:
    print('\n\n')
    print(generate_line(kanye,'one good girl',temp))




one good girl some the stop to the chool but i want to the stop to the chool but i want to the stop to the chool 



one good girl was to the caller i want to the world to me




one good girl seased to go at the thought i been work some to the stop to come




one good girl drivon for the tabe hem stectide




one good girl




one good girl ragoicent



In [9]:
for temp in temps:
    print('\n\n')
    print(generate_line(kanye,'i need you',temp))




i need you got the chool but i want to the chool but i want to the stop to the chool but i want to the stop to



i need you say now i know i aint said i want to stars to the stars the countor i know the cather the best of t



i need your money from the stop the money to the clob i aint come a break i can starting in the best all i kno



i need you got a swreak lets even back ot me the floobie headd my couble




i need you would leave it actring i




i need you himserfost



In [10]:
jcash = create_model('johnny-cash')

Epoch 1/5
1445/1445 [==============================] - 84s 56ms/step - loss: 2.4730 - accuracy: 0.2772
Epoch 2/5
1445/1445 [==============================] - 84s 58ms/step - loss: 2.0418 - accuracy: 0.3830
Epoch 3/5
1445/1445 [==============================] - 80s 55ms/step - loss: 1.8321 - accuracy: 0.4363
Epoch 4/5
1445/1445 [==============================] - 83s 57ms/step - loss: 1.6602 - accuracy: 0.4858
Epoch 5/5
1445/1445 [==============================] - 84s 58ms/step - loss: 1.5178 - accuracy: 0.5244


In [11]:
for temp in temps:
    print(generate_line(jcash,'im going to',temp))

im going to the dever love i was steed by the dever love i was steed by the dever love i was steed by the dever
im going to the steed by the devils singing the that the dear and the dever with the dear it a cauld the dight 
im going to me a true the deed ring

im going to take on my bany

im going to uved dong know

im going tolwwen



In [12]:
for temp in temps:
    print(generate_line(jcash,'and it burns burns burns',temp))

and it burns burns burns of the dear and the dever love i shall meet my bone

and it burns burns burns of the dont the dever meet my heart of the dear the des well the devils of the done

and it burns burns burns of him can is far

and it burns burns burns sumen and hise drem man dark

and it burns burns burns hereer but shut went the weud

and it burns burns burns day



In [13]:
for temp in temps:
    print(generate_line(jcash,'i shot a man',temp))

i shot a man i was a long the dever love i was steed by the dever love i shall meet my bone

i shot a man in the dever love in the right and the devine i was gonna couldnt be morning of the dear the the fe
i shot a mand my hald

i shot a man dont of with me heart let fur heavun if buens that about death and mahe you

i shot a manys rayy go shill though

i shot a man  zomeslingsdver

